## Подготовка моделей 

В данном ноутбуке будет производится подготовка моделей машинного обучения для прогнозирования. 

В рамках данной задачи будут протестированы следующие модели  машинного обучения. 

1. Линейная многофакторная регрессия для первичных экспериментов и оценки базовых параметров. 
2. Регуляризированные модели - для работы в условиях возможной мультиколлинеарности, и для работы по отбору признаков.
3. Деревья решениий - для анализа работы в условиях наличия  нелинейных зависимостей. 
4. Случайный лес - для задачи, если итоговый признаки будут иметь сложную взаимосвязь. 
5. Градиентный  бустинг - как один из самых мощных методов машинного обучения. 
разведочный анализ данных предлагаемого датасета, а также будет проводиться подготовка к формированию итогового датасета, который будет использоваться для итоговых моделей.

Также будут протестировано применение ансамблиевых методов регрессии - Stacking и Voting. 

## Подготовительный этап

В рамках данного этапа будут подготовлены данные с двумя наборами переменных. С целевой переменной и предикторами, произведена загрузка стандартных библиотек, а также проведена нормализация признаков.

In [123]:
## Импорт стандартных библиотек
import pandas as pd
import numpy as np

## Прочие библиотеки
import warnings
warnings.filterwarnings("ignore")

# Метрики
from sklearn import model_selection, metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

#Выбор моделей
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# Модели машинного обучения
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

# Ансамбли
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor

# Прочее 
from sklearn.preprocessing import StandardScaler

In [124]:
# Загрузка данных
df = pd.read_csv("Data/Processed/Final.csv")
df = pd.DataFrame(df)
df

,Поток Энергий,Древний Ветер,Дыхание Истока,Магическая производительность,Расхождения в стабильности магии,Гармония Бессмертия
0,6677.380,0.000,0.95,18.820976,-0.025,0.975
1,6828.469,1.019,0.95,10.754843,-0.025,0.975
2,7111.811,0.000,0.95,17.876293,-0.025,0.975
3,7792.630,0.000,0.95,17.243799,-0.025,0.975
4,8494.777,1.026,0.95,14.782759,-0.025,0.975
...,...,...,...,...,...,...
11929,8470.013,1.027,1.00,16.383439,0.000,1.000
11930,8800.352,0.000,1.00,14.312087,0.000,1.000
11931,9120.889,1.036,1.00,12.953525,0.000,1.000
11932,9300.274,0.000,1.00,10.745483,0.000,1.000


In [125]:
# Отделение целевой переменной от остальных
X = df.drop("Гармония Бессмертия", axis = 1 )
Y = df['Гармония Бессмертия']

In [126]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.2,
                                                    random_state=42)
st = StandardScaler()
#X_train_std = st.fit_transform(X_train)
#X_test_std = st.transform(X_test)
X_train = st.fit_transform(X_train)
X_test = st.transform(X_test)

Предварительные данные готовы. Можно приступать к построению моделей.

### Этап построения моделей. 

Зададим основные модели, которые будем рассмотривать в данном кейсе. 

In [127]:
model_LR = LinearRegression()
model_Ridge = Ridge()
model_Lasso = Lasso()
model_DTR = DecisionTreeRegressor()
model_GBR = GradientBoostingRegressor()
model_RFR = RandomForestRegressor()

В рамках кейса были выбраны следующие модели. 

- Линейная регрессия – как одна из самых простых и широко используемых моделей в машинном обучении. Модель используется в нашем случае для того, чтобы предллварительно оценить набор данных, их взаимосвязи и т.л. Не ожидается, что модель будет иметь адекватныые показатели качества. 

Модели регуляризации будут использоваться из-за того, что в изначальном датасете данные у нас были высококоррелирвоаны между собой. В нашем  случае мы также ожидаем, что это повлияет на финальный датасет. Использование моделей с регуляризацией позволит снизить это влияние. 

- Регрессия Ridge добавляет штраф за сложность модели путем добавления квадрата коэффициентов модели к функции потерь. Это помогает избежать переобучения, ограничивая размер коэффициентов.
- Lasso аналогична Ridge, но использует штраф за абсолютную величину коэффициентов , что приводит к тому, что некоторые коэффициенты становятся равными нулю. Это позволяет автоматически проводить отбор признаков.

Деревья решений используются для того, чтобы понять общий уровень качества будущей модели для применения более продвинутых ансамблиевыхх моделей, которые описаны ниже. 

- Деревья решений строят иерархию правил принятия решений, разбивая пространство признаков на области, в каждой из которых делается предсказание среднего значения целевой переменной. Для каждого узла дерева выбирается признак, который наилучшим образом разделяет данные на подмножества.

- Градиентный бустинг представляет собой ансамблевый метод, который последовательно строит множество слабых моделей, каждая из которых пытается исправить ошибки предыдущей модели. 

- Случайный лес состоит из множества деревьев решений, каждое из которых строится на случайной подвыборке данных и случайном подмножестве признаков. Прогнозирование осуществляется путем усреднения предсказаний всех деревьев.

Для более оптимального построения моделей создадим несколько функций. 
- функция загрузки данных в модель машинного обучения и вывод основных метрик. 
- проверка модели на уровень переобучениия. 
Данные функции далее будут объединены в одну функцию для организации более полного отчета о работе модели. 

In [128]:
# Модель для оценки качества моделей. На вход принимает саму модель и данные для обучения и теста
def model_work(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = mean_squared_error(y_test, predictions) ** 0.5
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print(f"Метрики модели: {model}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R^2: {r2}")

Ниже представлена функция, которая проверяет общий уровень переобучения. 

Алгоритм функции: 

1. Сначала модель делает прогнозы на основе обучающих данных (X_train) и тестовых данных (X_test).
2. Затем функция применяет переданную метрику к прогнозам и реальным значениям для обеих выборок (тренировочной и тестовой).
3. После этого выводятся результаты применения метрики к обеим выборкам: рассчитывается разница между значениями метрик на тренировочном и тестовом наборах данных, выраженная в процентах от значения на тесте.
4. Если значение дельты велико, то это может свидетельствовать о наличии переобучения.

metric - метрика, используемая для оценки точности модели. В нашем случае такой метрикой будет MAE.

In [129]:
def overfitting(model, X_train, y_train, X_test, y_test, metric):
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    value_train = metric(y_train, y_pred_train)
    value_test = metric(y_test, y_pred_test)
    delta = (abs(value_train - value_test)/value_test*100)

    print(f'{metric.__name__} train: %.3f' % value_train)
    print(f'{metric.__name__} test: %.3f' % value_test)
    print(f'delta = {(abs(value_train - value_test)/value_test*100):.1f} %')
    if delta > 10:
        return print("Модель склонна к переобучению")
    else:
        return print("Модель не склонна к переобучению")

In [130]:
def report(model, X_train, y_train, X_test, y_test):
    print(f"\nОтчет о метриках качества модели: {model}")
    model_work(model, X_train, y_train, X_test, y_test)
    print(f"\nОтчет о метриках переобучения модели: {model}\n")
    overfitting(model, X_train, y_train, X_test, y_test, mean_absolute_error)

Итоговая функция report готова. Функция будет выводить основную информацию по модели

Далее проверим последовательно все вышеописанные модели на нашем датасете.

In [131]:
report(model_LR, X_train, y_train, X_test, y_test)



Отчет о метриках качества модели: LinearRegression()
Метрики модели: LinearRegression()
RMSE: 2.220446049250313e-16
MAE: 2.220446049250313e-16
R^2: 1.0

Отчет о метриках переобучения модели: LinearRegression()

mean_absolute_error train: 0.000
mean_absolute_error test: 0.000
delta = 0.0 %
Модель не склонна к переобучению


In [132]:
report(model_Ridge, X_train, y_train, X_test, y_test)


Отчет о метриках качества модели: Ridge()
Метрики модели: Ridge()
RMSE: 7.170107085913626e-06
MAE: 6.161401499039515e-06
R^2: 0.9999990978047385

Отчет о метриках переобучения модели: Ridge()

mean_absolute_error train: 0.000
mean_absolute_error test: 0.000
delta = 1.5 %
Модель не склонна к переобучению


In [134]:
report(model_Lasso, X_train, y_train, X_test, y_test)


Отчет о метриках качества модели: Lasso()
Метрики модели: Lasso()
RMSE: 0.007550256984551614
MAE: 0.0065406189447756394
R^2: -0.00039779410217088973

Отчет о метриках переобучения модели: Lasso()

mean_absolute_error train: 0.006
mean_absolute_error test: 0.007
delta = 0.8 %
Модель не склонна к переобучению


In [135]:
report(model_DTR, X_train, y_train, X_test, y_test)


Отчет о метриках качества модели: DecisionTreeRegressor()
Метрики модели: DecisionTreeRegressor()
RMSE: 7.898006511391756e-17
MAE: 4.7441453084108074e-17
R^2: 1.0

Отчет о метриках переобучения модели: DecisionTreeRegressor()

mean_absolute_error train: 0.000
mean_absolute_error test: 0.000
delta = 29.4 %
Модель склонна к переобучению


In [136]:
report(model_GBR, X_train, y_train, X_test, y_test)


Отчет о метриках качества модели: GradientBoostingRegressor()
Метрики модели: GradientBoostingRegressor()
RMSE: 0.0006024453615377488
MAE: 0.00042065123596096726
R^2: 0.9936308043801235

Отчет о метриках переобучения модели: GradientBoostingRegressor()

mean_absolute_error train: 0.000
mean_absolute_error test: 0.000
delta = 5.4 %
Модель не склонна к переобучению


In [137]:
report(model_RFR, X_train, y_train, X_test, y_test)


Отчет о метриках качества модели: RandomForestRegressor()
Метрики модели: RandomForestRegressor()
RMSE: 6.086549346649736e-05
MAE: 3.681189777967306e-05
R^2: 0.9999349882954923

Отчет о метриках переобучения модели: RandomForestRegressor()

mean_absolute_error train: 0.000
mean_absolute_error test: 0.000
delta = 57.3 %
Модель склонна к переобучению


In [143]:
ensemble_stack = StackingRegressor([('lr', LinearRegression()),
                            ('GBR', GradientBoostingRegressor()),
                            ('Ridge', Ridge()),
                            ('Lasso', Lasso()),
                            ("DTR", DecisionTreeRegressor()),
                            ('RFR', RandomForestRegressor())])

In [144]:
ensemble_vote = VotingRegressor([('lr', LinearRegression()),
                            ('GBR', GradientBoostingRegressor()),
                            ('Ridge', Ridge()),
                            ('Lasso', Lasso()),
                            ("DTR", DecisionTreeRegressor()),
                            ('RFR', RandomForestRegressor())])

In [145]:
report(ensemble_stack, X_train, y_train, X_test, y_test)


Отчет о метриках качества модели: StackingRegressor(estimators=[('lr', LinearRegression()),
                              ('GBR', GradientBoostingRegressor()),
                              ('Ridge', Ridge()), ('Lasso', Lasso()),
                              ('DTR', DecisionTreeRegressor()),
                              ('RFR', RandomForestRegressor())])
Метрики модели: StackingRegressor(estimators=[('lr', LinearRegression()),
                              ('GBR', GradientBoostingRegressor()),
                              ('Ridge', Ridge()), ('Lasso', Lasso()),
                              ('DTR', DecisionTreeRegressor()),
                              ('RFR', RandomForestRegressor())])
RMSE: 0.00029561213749379993
MAE: 0.00022729503138867519
R^2: 0.9984664646857914

Отчет о метриках переобучения модели: StackingRegressor(estimators=[('lr', LinearRegression()),
                              ('GBR', GradientBoostingRegressor()),
                              ('Ridge', Ridge()), ('L

In [146]:
report(ensemble_vote, X_train, y_train, X_test, y_test)


Отчет о метриках качества модели: VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('GBR', GradientBoostingRegressor()),
                            ('Ridge', Ridge()), ('Lasso', Lasso()),
                            ('DTR', DecisionTreeRegressor()),
                            ('RFR', RandomForestRegressor())])
Метрики модели: VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('GBR', GradientBoostingRegressor()),
                            ('Ridge', Ridge()), ('Lasso', Lasso()),
                            ('DTR', DecisionTreeRegressor()),
                            ('RFR', RandomForestRegressor())])
RMSE: 0.0013233421344055769
MAE: 0.0011258476537072905
R^2: 0.9692678079359256

Отчет о метриках переобучения модели: VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('GBR', GradientBoostingRegressor()),
                            ('Ridge', Ridge()), ('Lasso', Lasso()),
           